# Enhanced Emotional Reasoning Steering with COT-Steering Framework

This notebook provides a comprehensive implementation of emotional reasoning steering for language models, extending the existing COT-steering framework to include **depressive-normal dichotomy** and enhanced emotional reasoning patterns.

## Key Enhancements

1. **Extended Training Data**: Added 40+ new emotional prompts across depressive, anxious, negative attribution, and pessimistic thinking patterns
2. **Normal Thinking Baseline**: Added 'normal-thinking' label for balanced, healthy reasoning patterns
3. **Depressive-Normal Dichotomy**: Emotional vectors are computed by subtracting normal-thinking vectors from negative emotional vectors
4. **Unified Pipeline**: New `emotional_steering_pipeline` function for streamlined emotional steering
5. **Enhanced Caching**: All steps are cached to enable resumable processing
6. **Google Colab Compatibility**: Optimized for Google Colab with proper mounting and caching

## ⚠️ Important Safety Notice

This implementation is intended for **research purposes only**. Steering models toward negative emotional states could be harmful if misused. Please:
- Use only for legitimate research with proper ethical oversight
- Always provide counterbalancing positive steering capabilities
- Never deploy this in production systems without appropriate safeguards
- Ensure users are aware when emotional steering is active


## Setup and Dependencies

In [ ]:
%%bash
set -e  # stop on first error

# ---------- 1) Miniconda (if not present) ----------
if ! command -v conda >/dev/null 2>&1; then
  echo "[INFO] Installing Miniconda to /usr/local ..."
  wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
  chmod +x Miniconda3-latest-Linux-x86_64.sh
  bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
fi

# Make conda usable in this shell
export PATH="/usr/local/bin:$PATH"
source /usr/local/etc/profile.d/conda.sh

# Speed up env solve
conda config --set channel_priority flexible
conda config --add channels conda-forge >/dev/null 2>&1 || true
conda install -n base -y mamba git pip

# ---------- 2) Clone repo (HTTPS instead of SSH) ----------
if [ ! -d "steering-thinking-llms" ]; then
  git clone https://github.com/cvenhoff/steering-thinking-llms.git
fi
cd steering-thinking-llms

# ---------- 3) Create & activate conda env ----------
# The repo’s README uses: conda env create -f environment.yaml  && conda activate stllms_env
# Use mamba for speed (falls back to conda if needed).
if command -v mamba >/dev/null 2>&1; then
  mamba env create -f environment.yaml
else
  conda env create -f environment.yaml
fi

# The environment name is defined in environment.yaml (README shows 'stllms_env')
conda activate stllms_env

# ---------- 4) Install package in editable mode ----------
pip install -e .

# ---------- 5) (Optional) Expose env as a Jupyter kernel for Colab ----------
python -m ipykernel install --user --name stllms_env --display-name "Python (stllms_env)"

echo "✅ Setup complete. Env 'stllms_env' created & package installed."

In [ ]:
# Install required packages if not already installed
!pip install torch transformers nnsight openai anthropic python-dotenv tqdm matplotlib seaborn pandas numpy
# !pip install -U bitsandbytes -U transformers

In [ ]:
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh

In [ ]:
%%bash
set -e  # stop on first error

# ---------- 1) Miniconda (if not present) ----------
if ! command -v conda >/dev/null 2>&1; then
  echo "[INFO] Installing Miniconda to /usr/local ..."
  wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
  chmod +x Miniconda3-latest-Linux-x86_64.sh
  bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
fi

# Make conda usable in this shell
export PATH="/usr/local/bin:$PATH"
source /usr/local/etc/profile.d/conda.sh

# Speed up env solve
conda config --set channel_priority flexible
conda config --add channels conda-forge >/dev/null 2>&1 || true
conda install -n base -y mamba git pip

# ---------- 2) Clone repo (HTTPS instead of SSH) ----------
if [ ! -d "steering-thinking-llms" ]; then
  git clone https://github.com/cvenhoff/steering-thinking-llms.git
fi
cd steering-thinking-llms

# ---------- 3) Create & activate conda env ----------
# The repo’s README uses: conda env create -f environment.yaml  && conda activate stllms_env
# Use mamba for speed (falls back to conda if needed).
if command -v mamba >/dev/null 2>&1; then
  mamba env create -f environment.yaml
else
  conda env create -f environment.yaml
fi

# The environment name is defined in environment.yaml (README shows 'stllms_env')
conda activate stllms_env

# ---------- 4) Install package in editable mode ----------
pip install -e .

# ---------- 5) (Optional) Expose env as a Jupyter kernel for Colab ----------
python -m ipykernel install --user --name stllms_env --display-name "Python (stllms_env)"

echo "✅ Setup complete. Env 'stllms_env' created & package installed."

In [ ]:
%%bash
set -e                       # stop on first error

# ---------- 1. Install Miniconda ---------- #
wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# ---------- 2. Prepare conda ---------- #
export PATH="/usr/local/bin:$PATH"
source /usr/local/etc/profile.d/conda.sh   # enables 'conda activate'

# ---------- 3. Clone repo (if needed) ---------- #
if [ ! -d "COT-steering" ]; then
  git clone https://github.com/ChuloIva/COT-steering.git
fi
cd COT-steering

# ---------- 4. Create env & install pkg ---------- #
conda env create -f environment.yaml         # creates stllms_env
conda activate stllms_env
pip install -e .

echo "✅ Finished: environment 'stllms_env' ready and COT-steering installed."

In [ ]:
# Google Colab specific setup
import os
import sys

# Check if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Clone the repository if not already present
    if not os.path.exists('./COT-steering'):
        !git clone https://github.com/ChuloIva/COT-steering
    
    os.chdir('./COT-steering')
    print("Current working directory:", os.getcwd())
    


In [ ]:
    # Optional: Link to Google Drive for persistent storage
#     DRIVE_PATH = '/content/drive/MyDrive/COT_Steering_Results'
#     if not os.path.exists(DRIVE_PATH):
#         os.makedirs(DRIVE_PATH)
#     print(f"Drive storage path: {DRIVE_PATH}")
# else:
#     print("Running locally")
#     DRIVE_PATH = './results'

In [ ]:
# Hugging Face login
from huggingface_hub import login

# Log in to Hugging Face Hub (set HUGGINGFACE_TOKEN env variable or paste when prompted)
login(token=None, add_to_git_credential=False)

In [ ]:
import sys
import os

# Add paths to import local modules
sys.path.append('./utils')
sys.path.append('./messages')

# Import required libraries
import torch
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
from utils import (
    load_model_and_vectors,
    process_batch_annotations,
    process_saved_responses_batch,
    custom_generate_steering,
    analyze_emotional_content,
    generate_and_analyze_emotional,
    emotional_steering_pipeline,
    steering_config,
    chat
)

from messages import messages, eval_messages

print("✅ Dependencies loaded successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"💾 CUDA available: {torch.cuda.is_available()}")


## Configuration and API Keys

In [ ]:
# API Keys (set these with your actual keys)
import os
os.environ['OPENAI_API_KEY'] = ''  # Add your OpenAI API key here
os.environ['ANTHROPIC_API_KEY'] = ''  # Add your Anthropic API key here

# Configuration settings
CONFIG = {
    "model_name": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",  # Change as needed
    "device": "auto",  # auto-detect, or specify "cuda", "mps", "cpu"
    "load_in_8bit": False,
    "max_new_tokens": 1000,
    "batch_size": 4,
    "include_emotional": True,  # Whether to include emotional reasoning in training
    "results_dir": DRIVE_PATH if IN_COLAB else "./results",
    "timestamp": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "enable_caching": True,  # Enable caching for resumable processing
    "cache_dir": os.path.join(DRIVE_PATH if IN_COLAB else "./results", "cache")
}

# Create directories
for dir_name in [CONFIG["results_dir"], CONFIG["cache_dir"], 
                 f"{CONFIG['results_dir']}/figures", f"{CONFIG['results_dir']}/data"]:
    os.makedirs(dir_name, exist_ok=True)

print(f"📋 Configuration:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

## Step 1: Load Model and Check for Cached Vectors

In [ ]:
def load_cached_data(config):
    """Load cached data if available"""
    cached_data = {}
    
    # Check for cached model vectors
    model_id = config["model_name"].split('/')[-1].lower()
    vector_files = [
        f"mean_vectors_{model_id}.pt",
        f"feature_vectors_{model_id}.pt",
        f"steering_vectors_{model_id}.pt"
    ]
    
    for filename in vector_files:
        filepath = os.path.join(config["cache_dir"], filename)
        if os.path.exists(filepath):
            try:
                cached_data[filename.replace('.pt', '')] = torch.load(filepath, map_location='cpu')
                print(f"✅ Loaded cached {filename}")
            except Exception as e:
                print(f"⚠️  Error loading {filename}: {e}")
    
    # Check for cached responses and annotations
    response_files = [
        "emotional_responses.json",
        "emotional_annotations.json",
        "training_results.json"
    ]
    
    for filename in response_files:
        filepath = os.path.join(config["cache_dir"], filename)
        if os.path.exists(filepath):
            try:
                with open(filepath, 'r') as f:
                    cached_data[filename.replace('.json', '')] = json.load(f)
                print(f"✅ Loaded cached {filename}")
            except Exception as e:
                print(f"⚠️  Error loading {filename}: {e}")
    
    return cached_data

# Load cached data
print("🔍 Checking for cached data...")
cached_data = load_cached_data(CONFIG)

if cached_data:
    print(f"📦 Found {len(cached_data)} cached items: {list(cached_data.keys())}")
else:
    print("📝 No cached data found - will generate from scratch")

In [ ]:
print("🤖 Loading model and tokenizer...")

model, tokenizer, existing_vectors = load_model_and_vectors(
    device=CONFIG["device"],
    load_in_8bit=CONFIG["load_in_8bit"],
    compute_features=True,
    model_name=CONFIG["model_name"]
)

print(f"✅ Model loaded: {CONFIG['model_name']}")
print(f"📊 Device: {next(model.parameters()).device}")
print(f"🎯 Model has {model.config.num_hidden_layers} layers")
print(f"📝 Vocabulary size: {len(tokenizer)}")

# Check for existing vectors or use cached ones
if 'feature_vectors' in cached_data:
    feature_vectors = cached_data['feature_vectors']
    print(f"📦 Using cached feature vectors: {list(feature_vectors.keys())}")
elif existing_vectors:
    feature_vectors = existing_vectors
    print(f"📦 Using existing feature vectors: {list(feature_vectors.keys())}")
else:
    feature_vectors = None
    print("⚠️  No feature vectors found - will need to train from scratch")

## Step 2: Prepare Enhanced Emotional Messages

The enhanced message set includes:
- **40+ additional emotional prompts** across all categories
- **15 normal/balanced thinking prompts** for the normal-thinking baseline
- **Better coverage** of depressive, anxious, negative attribution, and pessimistic patterns

In [ ]:
# Enhanced message filtering with better categorization
def categorize_messages(messages):
    """Categorize messages into emotional and cognitive types with enhanced detection"""
    emotional_messages = []
    cognitive_messages = []
    normal_messages = []
    
    for msg in messages:
        content = msg["content"].lower()
        
        # Enhanced emotional indicators
        emotional_indicators = [
            "you've been", "your recent", "everyone around you", "after receiving",
            "what does this", "how do you interpret", "what might this", "what are all the ways",
            "walk through all", "consider all the potential", "what could this",
            "what are the various", "what are the ways", "what challenges", "what obstacles",
            "how do you feel", "what thoughts are", "what does this suggest", "what might others",
            "how do you view", "what does this indicate", "what might this change",
            "what concerning", "what dangerous", "what problems", "what negative",
            "what factors contributed", "how do you account", "what motivated them",
            "what explains", "what accounted for"
        ]
        
        # Normal thinking indicators
        normal_indicators = [
            "how would you approach", "what factors would you", "what strategies would",
            "how would you use", "what would be a reasonable", "how would you best",
            "how would you evaluate", "what would be a healthy", "what considerations would",
            "how would you decide", "what steps would help", "what approach would help",
            "what process would help"
        ]
        
        if any(indicator in content for indicator in normal_indicators):
            normal_messages.append(msg)
        elif any(indicator in content for indicator in emotional_indicators):
            emotional_messages.append(msg)
        else:
            cognitive_messages.append(msg)
    
    return emotional_messages, cognitive_messages, normal_messages

emotional_messages, cognitive_messages, normal_messages = categorize_messages(messages)

print(f"📊 Enhanced Message Breakdown:")
print(f"   🧠 Cognitive messages: {len(cognitive_messages)}")
print(f"   😔 Emotional messages: {len(emotional_messages)}")
print(f"   🎯 Normal thinking messages: {len(normal_messages)}")
print(f"   📝 Total messages: {len(messages)}")

# Show examples
print(f"\n📝 Example emotional messages:")
for i, msg in enumerate(emotional_messages[:3]):
    print(f"   {i+1}. {msg['content'][:80]}...")

print(f"\n🎯 Example normal thinking messages:")
for i, msg in enumerate(normal_messages[:3]):
    print(f"   {i+1}. {msg['content'][:80]}...")

## Step 3: Generate or Load Training Responses

This step generates responses to emotional and normal thinking prompts, with caching support.

In [ ]:
def generate_or_load_responses(messages_subset, config, cache_key, max_new_tokens=1000):
    """Generate responses or load from cache"""
    cache_file = os.path.join(config["cache_dir"], f"{cache_key}.json")
    
    if config["enable_caching"] and os.path.exists(cache_file):
        print(f"📂 Loading cached responses from {cache_file}")
        with open(cache_file, 'r') as f:
            return json.load(f)
    
    print(f"🔄 Generating {len(messages_subset)} responses...")
    responses = []

    for msg in tqdm(messages_subset, desc="Generating responses"):
        try:
            input_ids = tokenizer.encode(msg["content"], return_tensors="pt")

            with model.generate(
                {"input_ids": input_ids, "attention_mask": (input_ids != tokenizer.pad_token_id).long()},
                max_new_tokens=max_new_tokens,
                pad_token_id=tokenizer.pad_token_id
            ) as tracer:
                output = model.generator.output.save()

            response_text = tokenizer.decode(output[0], skip_special_tokens=True)
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            if response_text.startswith(input_text):
                response_text = response_text[len(input_text):].strip()

            responses.append({
                "message": msg["content"],
                "response": response_text
            })

        except Exception as e:
            print(f"Error generating response: {e}")
            continue
    
    # Cache the results
    if config["enable_caching"]:
        with open(cache_file, 'w') as f:
            json.dump(responses, f, indent=2)
        print(f"💾 Cached responses to {cache_file}")
    
    return responses

# Generate responses for both emotional and normal messages
combined_training_messages = emotional_messages[:30] + normal_messages[:15]  # Balanced training set
training_responses = generate_or_load_responses(
    combined_training_messages, CONFIG, "enhanced_training_responses", CONFIG["max_new_tokens"]
)

print(f"✅ Ready with {len(training_responses)} training responses")

# Show example
if training_responses:
    print(f"\n📝 Example response:")
    print(f"   Input: {training_responses[0]['message'][:80]}...")
    print(f"   Output: {training_responses[0]['response'][:150]}...")

## Step 4: Annotate Responses with Enhanced Emotional Labels

This step uses GPT-4 to annotate responses with both cognitive and emotional reasoning labels, including the new **normal-thinking** label.

In [ ]:
def generate_or_load_annotations(responses, config, cache_key, include_emotional=True):
    """Generate annotations or load from cache"""
    cache_file = os.path.join(config["cache_dir"], f"{cache_key}.json")
    
    if config["enable_caching"] and os.path.exists(cache_file):
        print(f"📂 Loading cached annotations from {cache_file}")
        with open(cache_file, 'r') as f:
            annotation_data = json.load(f)
            return [item["annotation"] for item in annotation_data["responses"]]
    
    print(f"🏷️  Generating annotations for {len(responses)} responses...")
    response_texts = [resp["response"] for resp in responses]
    
    # Generate annotations
    annotated_responses = process_batch_annotations(
        response_texts, include_emotional=include_emotional
    )
    
    # Cache the results
    if config["enable_caching"]:
        annotation_data = {
            "timestamp": config["timestamp"],
            "model_name": config["model_name"],
            "include_emotional": include_emotional,
            "responses": [
                {
                    "message": responses[i]["message"],
                    "response": responses[i]["response"],
                    "annotation": annotated_responses[i]
                }
                for i in range(len(responses))
            ]
        }
        
        with open(cache_file, "w") as f:
            json.dump(annotation_data, f, indent=2)
        print(f"💾 Cached annotations to {cache_file}")
    
    return annotated_responses

# Generate annotations with enhanced emotional labels
annotated_responses = generate_or_load_annotations(
    training_responses, CONFIG, "enhanced_annotations", include_emotional=True
)

print(f"✅ Generated {len(annotated_responses)} annotations")

# Show example annotation
if annotated_responses:
    print(f"\n📝 Example annotation:")
    print(f"   Original: {training_responses[0]['response'][:100]}...")
    print(f"   Annotated: {annotated_responses[0][:200]}...")

## Step 5: Train Enhanced Emotional Vectors

This step processes the annotated responses to extract neural activations and train steering vectors using the **depressive-normal dichotomy** approach.

In [ ]:
def process_saved_responses_batch_optimized(responses_list, tokenizer, model, keep_on_gpu=True, batch_size=8):
    """Optimized version with GPU utilization and progress tracking"""
    device = next(model.parameters()).device
    all_batch_outputs = []

    # Process in smaller batches to manage memory
    for batch_start in tqdm(range(0, len(responses_list), batch_size), desc="Processing response batches"):
        batch_end = min(batch_start + batch_size, len(responses_list))
        batch_responses = responses_list[batch_start:batch_end]

        tokenized_responses = get_batched_message_ids(tokenizer, batch_responses, device.type)

        # Process the inputs through the model to get activations
        layer_outputs = []
        with model.trace({
            "input_ids": tokenized_responses,
            "attention_mask": (tokenized_responses != tokenizer.pad_token_id).long()
        }) as tracer:
            # Capture layer outputs with progress
            for layer_idx in tqdm(range(model.config.num_hidden_layers),
                                desc=f"Extracting layers (batch {batch_start//batch_size + 1})",
                                leave=False):
                layer_outputs.append(model.model.layers[layer_idx].output[0].save())

        # Keep on GPU if requested, otherwise move to CPU
        if keep_on_gpu and device.type == 'cuda':
            layer_outputs = [x.value.detach() for x in layer_outputs]  # Keep on GPU
        else:
            layer_outputs = [x.value.cpu().detach().to(torch.float32) for x in layer_outputs]

        batch_layer_outputs = []

        for batch_idx in tqdm(range(len(batch_responses)),
                            desc=f"Processing examples (batch {batch_start//batch_size + 1})",
                            leave=False):
            # get length of padding tokens
            attention_mask = (tokenized_responses[batch_idx] != tokenizer.pad_token_id).long()
            padding_length = (attention_mask.squeeze() == 0).sum().item()

            # Slice out just the non-padded activations for this example across all layers
            example_outputs = torch.stack([
                layer_output[batch_idx][padding_length:]
                for layer_output in layer_outputs
            ])

            batch_layer_outputs.append(example_outputs)

        all_batch_outputs.extend(batch_layer_outputs)

        # Clear GPU cache between batches
        if device.type == 'cuda':
            torch.cuda.empty_cache()

    return all_batch_outputs

def train_enhanced_emotional_vectors_optimized(responses, annotations, model, tokenizer, config):
    """Optimized training with better GPU utilization and progress tracking"""

    cache_file = os.path.join(config["cache_dir"], "enhanced_mean_vectors.pt")

    if config["enable_caching"] and os.path.exists(cache_file):
        print(f"📂 Loading cached mean vectors from {cache_file}")
        return torch.load(cache_file, map_location='cpu')

    print("🧠 Training enhanced emotional vectors with GPU optimization...")
    device = next(model.parameters()).device

    # Extract activations with optimized function
    print("   📊 Extracting neural activations...")
    batch_activations = process_saved_responses_batch_optimized(
        responses, tokenizer, model,
        keep_on_gpu=(device.type == 'cuda'),
        batch_size=4  # Adjust based on GPU memory
    )

    # Initialize mean vectors storage
    from collections import defaultdict
    mean_vectors = defaultdict(lambda: {
        'mean': torch.zeros(model.config.num_hidden_layers, model.config.hidden_size, device=device),
        'count': 0
    })

    # Process annotations to find labels and compute means with progress
    print("   🏷️  Processing annotations and computing means...")
    successful_processed = 0

    for i, (response, annotation) in enumerate(tqdm(zip(responses, annotations),
                                                   total=len(responses),
                                                   desc="Computing mean vectors")):
        try:
            # Extract label positions
            from utils import get_label_positions
            label_positions = get_label_positions(annotation, response, tokenizer)

            # Get activations for this response
            if i < len(batch_activations):
                activations = batch_activations[i]

                # Move to GPU for computation if not already there
                if device.type == 'cuda' and activations.device.type != 'cuda':
                    activations = activations.to(device)

                # Ensure activation tensor has correct shape
                if len(activations.shape) == 2 and activations.shape[0] == model.config.num_hidden_layers:
                    # Update overall mean using running average
                    current_count = mean_vectors['overall']['count']
                    current_mean = mean_vectors['overall']['mean']
                    mean_vectors['overall']['mean'] = current_mean + (activations - current_mean) / (current_count + 1)
                    mean_vectors['overall']['count'] += 1

                    # Update label-specific means
                    for label in label_positions.keys():
                        if label != 'end-section':
                            current_count = mean_vectors[label]['count']
                            current_mean = mean_vectors[label]['mean']
                            mean_vectors[label]['mean'] = current_mean + (activations - current_mean) / (current_count + 1)
                            mean_vectors[label]['count'] += 1

                    successful_processed += 1

        except Exception as e:
            print(f"   ⚠️ Error processing response {i}: {e}")
            continue

        # Periodic progress update
        if (i + 1) % 10 == 0:
            print(f"   📈 Processed {i+1}/{len(responses)} responses, {successful_processed} successful")

    # Convert to CPU for saving and create regular dict
    print("   💾 Converting results and caching...")
    save_dict = {}
    for k, v in tqdm(mean_vectors.items(), desc="Converting to save format"):
        save_dict[k] = {
            'mean': v['mean'].cpu(),
            'count': v['count']
        }

    # Cache the results
    if config["enable_caching"]:
        torch.save(save_dict, cache_file)
        print(f"💾 Cached mean vectors to {cache_file}")

    print(f"✅ Training completed! Processed {successful_processed}/{len(responses)} responses successfully")
    print(f"🎯 Trained vectors for {len(save_dict)} categories:")
    for label, data in save_dict.items():
        print(f"   {label}: {data['count']} samples")

    return save_dict

In [ ]:
# FIXED Step 5: Train Enhanced Emotional Vectors (GPU Optimized)

# First, let's check your actual device setup
print("🔍 Device Detection:")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   CUDA device count: {torch.cuda.device_count()}")
    print(f"   Current CUDA device: {torch.cuda.current_device()}")
    print(f"   CUDA device name: {torch.cuda.get_device_name()}")

print(f"   MPS available: {hasattr(torch.backends, 'mps') and torch.backends.mps.is_available()}")
print(f"   Model device: {next(model.parameters()).device}")

def train_enhanced_emotional_vectors_optimized_fixed(responses, annotations, model, tokenizer, config):
    """Train steering vectors with enhanced emotional categories and normal baseline - FIXED"""

    cache_file = os.path.join(config["cache_dir"], "enhanced_mean_vectors.pt")

    if config["enable_caching"] and os.path.exists(cache_file):
        print(f"📂 Loading cached mean vectors from {cache_file}")
        return torch.load(cache_file, map_location='cpu')

    print("🧠 Training enhanced emotional vectors with GPU optimization...")
    device = next(model.parameters()).device
    print(f"   🎯 Using device: {device}")

    # Import the function from utils - this fixes the NameError
    from utils import get_batched_message_ids

    def process_responses_gpu_optimized_fixed(responses_list, batch_size=4):
        """Process responses keeping activations on GPU for faster computation - FIXED"""
        all_activations = []

        for batch_start in tqdm(range(0, len(responses_list), batch_size), desc="🔄 Processing batches"):
            batch_end = min(batch_start + batch_size, len(responses_list))
            batch_responses = responses_list[batch_start:batch_end]

            # Tokenize batch - use the actual device string, not device.type
            device_str = str(device).split(':')[0]  # Convert 'cuda:0' to 'cuda'
            tokenized_responses = get_batched_message_ids(tokenizer, batch_responses, device_str)

            # Ensure tokenized responses are on the same device as model
            if tokenized_responses.device != device:
                tokenized_responses = tokenized_responses.to(device)

            # Extract activations
            layer_outputs = []
            with model.trace({
                "input_ids": tokenized_responses,
                "attention_mask": (tokenized_responses != tokenizer.pad_token_id).long()
            }) as tracer:
                for layer_idx in tqdm(range(model.config.num_hidden_layers),
                                    desc=f"Extracting layers",
                                    leave=False):
                    layer_outputs.append(model.model.layers[layer_idx].output[0].save())

            # Keep on GPU for faster processing if using CUDA
            if device.type == 'cuda':
                layer_outputs = [x.value.detach() for x in layer_outputs]
                print(f"   ✅ Keeping batch {batch_start//batch_size + 1} on GPU")
            else:
                layer_outputs = [x.value.cpu().detach().to(torch.float32) for x in layer_outputs]
                print(f"   📱 Processing batch {batch_start//batch_size + 1} on {device}")

            # Process each example in batch - FIXED TENSOR SHAPE HANDLING
            batch_activations = []
            for batch_idx in tqdm(range(len(batch_responses)),
                                desc=f"Processing examples",
                                leave=False):
                attention_mask = (tokenized_responses[batch_idx] != tokenizer.pad_token_id).long()
                padding_length = (attention_mask.squeeze() == 0).sum().item()

                # Extract non-padded activations - CRITICAL FIX HERE
                example_activations = []
                for layer_output in layer_outputs:
                    # layer_output has shape [batch_size, seq_len, hidden_size]
                    # We want [seq_len, hidden_size] for this specific example, removing padding
                    layer_activation = layer_output[batch_idx][padding_length:]  # [actual_seq_len, hidden_size]
                    example_activations.append(layer_activation)
                
                # Stack to get [num_layers, actual_seq_len, hidden_size]
                example_outputs = torch.stack(example_activations, dim=0)
                
                # 🔧 CRITICAL FIX: Take mean across sequence dimension to get [num_layers, hidden_size]
                if len(example_outputs.shape) == 3:  # [num_layers, seq_len, hidden_size]
                    example_outputs = example_outputs.mean(dim=1)  # [num_layers, hidden_size]
                
                # Debug print for first few examples
                if batch_start == 0 and batch_idx < 2:
                    print(f"   🔍 Example {batch_idx} final shape: {example_outputs.shape}")
                    print(f"       Expected: [{model.config.num_hidden_layers}, {model.config.hidden_size}]")
                
                # Validate final shape
                if example_outputs.shape != (model.config.num_hidden_layers, model.config.hidden_size):
                    print(f"   ⚠️ Unexpected final shape for example {batch_idx}: {example_outputs.shape}")
                    print(f"       Expected: ({model.config.num_hidden_layers}, {model.config.hidden_size})")
                    continue
                
                batch_activations.append(example_outputs)

            all_activations.extend(batch_activations)

            # Clear cache periodically
            if device.type == 'cuda':
                torch.cuda.empty_cache()

            print(f"   📊 Completed batch {batch_start//batch_size + 1}/{(len(responses_list) + batch_size - 1)//batch_size}")

        return all_activations

    # Extract activations with GPU optimization
    print("   📊 Extracting neural activations...")
    batch_activations = process_responses_gpu_optimized_fixed(responses, batch_size=2)  # Smaller batch size for stability

    # Initialize mean vectors on appropriate device
    from collections import defaultdict

    # Use CPU for mean vector storage to avoid GPU memory issues
    mean_vectors = defaultdict(lambda: {
        'mean': torch.zeros(model.config.num_hidden_layers, model.config.hidden_size, dtype=torch.float32),
        'count': 0
    })

    # Process with detailed progress tracking - FIXED TENSOR OPERATIONS
    print("   🏷️  Computing mean vectors...")
    successful_processed = 0
    label_counts = defaultdict(int)

    for i, (response, annotation) in enumerate(tqdm(zip(responses, annotations),
                                                   total=len(responses),
                                                   desc="Computing means")):
        try:
            # Extract label positions
            from utils import get_label_positions
            label_positions = get_label_positions(annotation, response, tokenizer)

            if i < len(batch_activations) and label_positions:
                activations = batch_activations[i]

                # Move to CPU for mean computation to avoid GPU memory issues
                if activations.device.type != 'cpu':
                    activations = activations.cpu().float()

                # 🔧 CRITICAL FIX: Activations should already be [num_layers, hidden_size]
                # No need to take mean again - that was causing the dimension confusion!
                
                # Validate tensor dimensions before processing
                if activations.shape != (model.config.num_hidden_layers, model.config.hidden_size):
                    print(f"   ⚠️ Skipping response {i}: unexpected shape {activations.shape}")
                    print(f"       Expected: ({model.config.num_hidden_layers}, {model.config.hidden_size})")
                    continue

                # Update overall mean using incremental averaging
                current_count = mean_vectors['overall']['count']
                if current_count == 0:
                    mean_vectors['overall']['mean'] = activations.clone()
                else:
                    current_mean = mean_vectors['overall']['mean']
                    # Ensure tensor types match before arithmetic
                    if current_mean.dtype != activations.dtype:
                        activations = activations.to(current_mean.dtype)
                    mean_vectors['overall']['mean'] = current_mean + (activations - current_mean) / (current_count + 1)
                mean_vectors['overall']['count'] += 1

                # Update label-specific means
                for label in label_positions.keys():
                    if label != 'end-section':
                        current_count = mean_vectors[label]['count']
                        if current_count == 0:
                            mean_vectors[label]['mean'] = activations.clone()
                        else:
                            current_mean = mean_vectors[label]['mean']
                            # Ensure tensor types match
                            if current_mean.dtype != activations.dtype:
                                activations = activations.to(current_mean.dtype)
                            mean_vectors[label]['mean'] = current_mean + (activations - current_mean) / (current_count + 1)
                        mean_vectors[label]['count'] += 1
                        label_counts[label] += 1

                successful_processed += 1

        except Exception as e:
            print(f"   ⚠️ Error processing response {i}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue

        # Progress updates every 10 items
        if (i + 1) % 10 == 0:
            print(f"   📈 Progress: {i+1}/{len(responses)} | Successful: {successful_processed} | Labels found: {len(label_counts)}")
            # Show top 3 labels found so far
            if label_counts:
                top_labels = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)[:3]
                print(f"       Top labels: {dict(top_labels)}")

    # Convert to save format
    print("   💾 Preparing results for caching...")
    save_dict = {}
    for k, v in tqdm(mean_vectors.items(), desc="Converting tensors"):
        save_dict[k] = {
            'mean': v['mean'].clone(),
            'count': v['count']
        }

    # Cache results
    if config["enable_caching"]:
        torch.save(save_dict, cache_file)
        print(f"💾 Cached mean vectors to {cache_file}")

    print(f"\n✅ Training completed successfully!")
    print(f"📊 Final Statistics:")
    print(f"   Total responses processed: {successful_processed}/{len(responses)}")
    print(f"   Categories with training data: {len([k for k, v in save_dict.items() if v['count'] > 0])}")
    print(f"   Device used: {device}")

    print(f"\n📋 Categories and sample counts:")
    for label, data in sorted(save_dict.items(), key=lambda x: x[1]['count'], reverse=True):
        if data['count'] > 0:
            print(f"   ✅ {label}: {data['count']} samples")
        else:
            print(f"   ❌ {label}: {data['count']} samples")

    return save_dict

# Run the FIXED optimized training
enhanced_mean_vectors = train_enhanced_emotional_vectors_optimized_fixed(
    [r["response"] for r in training_responses],
    annotated_responses,
    model,
    tokenizer,
    CONFIG
)

🔍 Device Detection:
   CUDA available: True
   CUDA device count: 6
   Current CUDA device: 0
   CUDA device name: NVIDIA A100 80GB PCIe
   MPS available: False
   Model device: cuda:0
🧠 Training enhanced emotional vectors with GPU optimization...
   🎯 Using device: cuda:0
   📊 Extracting neural activations...
   ✅ Keeping batch 1 on GPU
   📊 Completed batch 1/22
   ✅ Keeping batch 2 on GPU
   📊 Completed batch 2/22
   ✅ Keeping batch 3 on GPU
   📊 Completed batch 3/22
   ✅ Keeping batch 4 on GPU
   📊 Completed batch 4/22
   ✅ Keeping batch 5 on GPU
   📊 Completed batch 5/22
   ✅ Keeping batch 6 on GPU
   📊 Completed batch 6/22
   ✅ Keeping batch 7 on GPU
   📊 Completed batch 7/22
   ✅ Keeping batch 8 on GPU
   📊 Completed batch 8/22
   ✅ Keeping batch 9 on GPU
   📊 Completed batch 9/22
   ✅ Keeping batch 10 on GPU
   📊 Completed batch 10/22
   ✅ Keeping batch 11 on GPU
   📊 Completed batch 11/22
   ✅ Keeping batch 12 on GPU
   📊 Completed batch 12/22
   ✅ Keeping batch 13 on GPU
   📊 Completed batch 13/22
   ✅ Keeping batch 14 on GPU
   📊 Completed batch 14/22
   ✅ Keeping batch 15 on GPU
   📊 Completed batch 15/22
   ✅ Keeping batch 16 on GPU
   📊 Completed batch 16/22
   ✅ Keeping batch 17 on GPU
   📊 Completed batch 17/22
   ✅ Keeping batch 18 on GPU
   📊 Completed batch 18/22
   ✅ Keeping batch 19 on GPU
   📊 Completed batch 19/22
   ✅ Keeping batch 20 on GPU
   📊 Completed batch 20/22
   ✅ Keeping batch 21 on GPU
   📊 Completed batch 21/22
   ✅ Keeping batch 22 on GPU
   📊 Completed batch 22/22
   🏷️  Computing mean vectors...
   ⚠️ Error processing response 2: The size of tensor a (4096) must match the size of tensor b (3810) at non-singleton dimension 1...
   💾 Preparing results for caching...
💾 Cached mean vectors to ./results/cache/enhanced_mean_vectors.pt

✅ Training completed successfully!
📊 Final Statistics:
   Total responses processed: 1/44
   Categories with training data: 3
   Device used: cuda:0

📋 Categories and sample counts:
   ✅ overall: 1 samples
   ✅ normal-thinking: 1 samples
   ✅ pessimistic-projection: 1 samples

In [ ]:
# Not used (code above is used)

# def train_enhanced_emotional_vectors(responses, annotations, model, tokenizer, config):
#     """Train steering vectors with enhanced emotional categories and normal baseline"""
    
#     cache_file = os.path.join(config["cache_dir"], "enhanced_mean_vectors.pt")
    
#     if config["enable_caching"] and os.path.exists(cache_file):
#         print(f"📂 Loading cached mean vectors from {cache_file}")
#         return torch.load(cache_file, map_location='cpu')
    
#     print("🧠 Training enhanced emotional vectors...")
    
#     # Extract activations
#     print("   Extracting neural activations...")
#     batch_activations = process_saved_responses_batch(responses, tokenizer, model)
    
#     # Initialize mean vectors storage
#     from collections import defaultdict
#     mean_vectors = defaultdict(lambda: {
#         'mean': torch.zeros(model.config.num_hidden_layers, model.config.hidden_size),
#         'count': 0
#     })
    
#     # Process annotations to find labels and compute means
#     print("   Processing annotations and computing means...")
#     for i, (response, annotation) in enumerate(tqdm(zip(responses, annotations), desc="Processing")):
#         try:
#             # Extract label positions
#             from utils import get_label_positions
#             label_positions = get_label_positions(annotation, response, tokenizer)
            
#             # Get activations for this response
#             if i < len(batch_activations):
#                 activations = batch_activations[i]
                
#                 # Ensure activation tensor has correct shape
#                 if len(activations.shape) == 2 and activations.shape[0] == model.config.num_hidden_layers:
#                     # Update overall mean
#                     current_count = mean_vectors['overall']['count']
#                     current_mean = mean_vectors['overall']['mean']
#                     mean_vectors['overall']['mean'] = current_mean + (activations - current_mean) / (current_count + 1)
#                     mean_vectors['overall']['count'] += 1
                    
#                     # Update label-specific means
#                     for label in label_positions.keys():
#                         if label != 'end-section':
#                             current_count = mean_vectors[label]['count']
#                             current_mean = mean_vectors[label]['mean']
#                             mean_vectors[label]['mean'] = current_mean + (activations - current_mean) / (current_count + 1)
#                             mean_vectors[label]['count'] += 1
        
#         except Exception as e:
#             print(f"   Error processing response {i}: {e}")
#             continue
    
#     # Convert to regular dict for saving
#     save_dict = {k: {'mean': v['mean'], 'count': v['count']} for k, v in mean_vectors.items()}
    
#     # Cache the results
#     if config["enable_caching"]:
#         torch.save(save_dict, cache_file)
#         print(f"💾 Cached mean vectors to {cache_file}")
    
#     print(f"✅ Trained vectors for {len(save_dict)} categories")
#     for label, data in save_dict.items():
#         print(f"   {label}: {data['count']} samples")
    
#     return save_dict

# # Train the enhanced emotional vectors
# enhanced_mean_vectors = train_enhanced_emotional_vectors(
#     [r["response"] for r in training_responses],
#     annotated_responses,
#     model,
#     tokenizer,
#     CONFIG
# )

## Step 6: Compute Enhanced Feature Vectors with Depressive-Normal Dichotomy

In [ ]:
def compute_enhanced_feature_vectors(mean_vectors_dict, config):
    """Compute feature vectors using the depressive-normal dichotomy approach"""
    
    cache_file = os.path.join(config["cache_dir"], "enhanced_feature_vectors.pt")
    
    if config["enable_caching"] and os.path.exists(cache_file):
        print(f"📂 Loading cached feature vectors from {cache_file}")
        return torch.load(cache_file, map_location='cpu')
    
    print("🧮 Computing enhanced feature vectors with depressive-normal dichotomy...")
    
    feature_vectors = {}
    
    # Check if we have normal-thinking vectors to use as baseline
    if "normal-thinking" in mean_vectors_dict:
        baseline_mean = mean_vectors_dict["normal-thinking"]["mean"]
        baseline_count = mean_vectors_dict["normal-thinking"]["count"]
        print(f"✅ Using normal-thinking as baseline ({baseline_count} samples)")
    elif "overall" in mean_vectors_dict:
        baseline_mean = mean_vectors_dict["overall"]["mean"]
        baseline_count = mean_vectors_dict["overall"]["count"]
        print(f"⚠️  Using overall mean as baseline ({baseline_count} samples)")
    else:
        print("❌ No baseline vectors available")
        return {}
    
    # Add baseline to feature vectors
    feature_vectors["baseline"] = baseline_mean
    
    # Compute differential vectors for emotional categories
    emotional_labels = ["depressive-thinking", "anxious-thinking", "negative-attribution", "pessimistic-projection"]
    
    for label in emotional_labels:
        if label in mean_vectors_dict:
            label_mean = mean_vectors_dict[label]["mean"]
            label_count = mean_vectors_dict[label]["count"]
            
            # Compute difference from baseline (normal thinking)
            feature_vectors[label] = label_mean - baseline_mean
            print(f"✅ Computed {label} feature vector ({label_count} samples)")
        else:
            print(f"⚠️  No data found for {label}")
    
    # Also include cognitive labels (use overall mean as baseline)
    if "overall" in mean_vectors_dict:
        overall_mean = mean_vectors_dict["overall"]["mean"]
        cognitive_labels = ["initializing", "deduction", "adding-knowledge", "example-testing", "uncertainty-estimation", "backtracking"]
        
        for label in cognitive_labels:
            if label in mean_vectors_dict:
                label_mean = mean_vectors_dict[label]["mean"]
                feature_vectors[label] = label_mean - overall_mean
                print(f"✅ Computed cognitive {label} feature vector")
    
    # Cache the results
    if config["enable_caching"]:
        torch.save(feature_vectors, cache_file)
        print(f"💾 Cached feature vectors to {cache_file}")
    
    print(f"\n🎯 Enhanced feature vectors ready:")
    for label in feature_vectors.keys():
        if label != "baseline":
            print(f"   {label}")
    
    return feature_vectors

# Compute enhanced feature vectors
enhanced_feature_vectors = compute_enhanced_feature_vectors(enhanced_mean_vectors, CONFIG)

# Use these as our main feature vectors for steering
if enhanced_feature_vectors:
    feature_vectors = enhanced_feature_vectors
    print(f"\n🚀 Ready for enhanced emotional steering!")
else:
    print(f"⚠️  No enhanced feature vectors available")

## Step 7: Test Enhanced Emotional Steering Pipeline

Test the new unified emotional steering pipeline with the **depressive-normal dichotomy**.

In [ ]:
# Test the enhanced emotional steering pipeline
if feature_vectors and len(feature_vectors) > 1:
    # Select test messages
    test_messages = emotional_messages[-5:]  # Use last 5 emotional messages for testing
    
    print(f"🧪 Testing enhanced emotional steering pipeline...")
    print(f"📝 Test messages: {len(test_messages)}")
    
    # Test depressive-normal dichotomy
    if "depressive-thinking" in feature_vectors and "normal-thinking" in feature_vectors:
        print("\n🎭 Testing Depressive-Normal Dichotomy")
        
        depressive_results = emotional_steering_pipeline(
            model=model,
            tokenizer=tokenizer,
            feature_vectors=feature_vectors,
            steering_config=steering_config,
            messages=test_messages,
            target_emotional_direction="depressive-normal",
            max_new_tokens=300,
            batch_size=2
        )
        
        # Save results
        results_file = os.path.join(CONFIG["results_dir"], f"depressive_normal_results_{CONFIG['timestamp']}.json")
        with open(results_file, 'w') as f:
            json.dump(depressive_results, f, indent=2)
        print(f"💾 Saved results to {results_file}")
        
    else:
        print("⚠️  Depressive-normal vectors not available for testing")
    
    # Test anxious-normal dichotomy if available
    if "anxious-thinking" in feature_vectors:
        print("\n🎭 Testing Anxious-Normal Dichotomy")
        
        anxious_results = emotional_steering_pipeline(
            model=model,
            tokenizer=tokenizer,
            feature_vectors=feature_vectors,
            steering_config=steering_config,
            messages=test_messages[:3],  # Smaller test set
            target_emotional_direction="anxious-normal",
            max_new_tokens=300,
            batch_size=2
        )
        
        # Save results
        results_file = os.path.join(CONFIG["results_dir"], f"anxious_normal_results_{CONFIG['timestamp']}.json")
        with open(results_file, 'w') as f:
            json.dump(anxious_results, f, indent=2)
        print(f"💾 Saved results to {results_file}")
    
else:
    print("⚠️  Insufficient feature vectors for pipeline testing")

## Step 8: Demonstrate Individual Steering Examples

Show specific examples of how the enhanced emotional steering works.

In [ ]:
def demonstrate_enhanced_steering(model, tokenizer, feature_vectors, steering_config, config):
    """Demonstrate enhanced emotional steering with specific examples"""
    
    if not feature_vectors or len(feature_vectors) < 2:
        print("⚠️  Insufficient feature vectors for demonstration")
        return
    
    demo_messages = [
        "You've been working on a personal project for weeks but haven't made much progress. How do you feel about your abilities?",
        "You have an important presentation tomorrow. What thoughts are going through your mind?",
        "You received some feedback on your work. How do you interpret this feedback?"
    ]
    
    print("🎭 Enhanced Emotional Steering Demonstration")
    print("=" * 60)
    
    available_labels = [label for label in feature_vectors.keys() if label != "baseline"]
    emotional_labels = [label for label in available_labels if "thinking" in label]
    
    print(f"🎯 Available emotional labels: {emotional_labels}")
    
    for i, message in enumerate(demo_messages[:min(len(demo_messages), len(emotional_labels))]):
        label = emotional_labels[i % len(emotional_labels)]
        model_name = config["model_name"]
        
        if model_name not in steering_config or label not in steering_config[model_name]:
            print(f"⚠️  Skipping {label} - steering config not available")
            continue
        
        print(f"\n📝 Message {i+1}: {message}")
        print(f"🎯 Demonstrating {label.replace('-', ' ').title()} steering")
        print("-" * 40)
        
        try:
            # Baseline response
            input_ids = tokenizer.encode(message, return_tensors="pt")
            
            with model.generate(
                {"input_ids": input_ids, "attention_mask": (input_ids != tokenizer.pad_token_id).long()},
                max_new_tokens=200,
                pad_token_id=tokenizer.pad_token_id
            ) as tracer:
                baseline_output = model.generator.output.save()
            
            baseline_text = tokenizer.decode(baseline_output[0], skip_special_tokens=True)
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            if baseline_text.startswith(input_text):
                baseline_text = baseline_text[len(input_text):].strip()
            
            baseline_analysis = analyze_emotional_content(baseline_text)
            
            print(f"🔵 Baseline Response:")
            print(f"   {baseline_text[:250]}...")
            print(f"   Emotional Score: {baseline_analysis['total_emotional_score']:.1f}%")
            
            # Enhanced steering (toward negative pattern)
            enhanced_result = generate_and_analyze_emotional(
                model, tokenizer, message, feature_vectors, steering_config,
                label, "positive", 200
            )
            
            print(f"\n🔴 Enhanced {label.replace('-', ' ').title()}:")
            print(f"   {enhanced_result['response'][:250]}...")
            print(f"   Emotional Score: {enhanced_result['emotional_analysis']['total_emotional_score']:.1f}%")
            
            # Suppressed steering (toward normal pattern)
            if "normal-thinking" in feature_vectors:
                suppressed_result = generate_and_analyze_emotional(
                    model, tokenizer, message, feature_vectors, steering_config,
                    "normal-thinking", "positive", 200
                )
                
                print(f"\n🟢 Enhanced Normal Thinking:")
                print(f"   {suppressed_result['response'][:250]}...")
                print(f"   Emotional Score: {suppressed_result['emotional_analysis']['total_emotional_score']:.1f}%")
            
            # Show steering effectiveness
            baseline_score = baseline_analysis['total_emotional_score']
            enhanced_score = enhanced_result['emotional_analysis']['total_emotional_score']
            
            effectiveness = abs(enhanced_score - baseline_score)
            print(f"\n📊 Steering Effectiveness: {effectiveness:.1f}% change")
            
        except Exception as e:
            print(f"❌ Error in demonstration for {label}: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    print(f"\n✅ Enhanced steering demonstration completed!")

# Run the enhanced demonstration
demonstrate_enhanced_steering(model, tokenizer, feature_vectors, steering_config, CONFIG)

## Step 9: Create Enhanced Visualizations

In [ ]:
def create_enhanced_visualizations(results_dict, config):
    """Create visualizations for enhanced emotional steering results"""
    
    if not results_dict or "overall_stats" not in results_dict:
        print("⚠️  No results available for visualization")
        return
    
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Enhanced Emotional Reasoning Steering Results', fontsize=16, fontweight='bold')
    
    stats = results_dict["overall_stats"]
    results = results_dict["results"]
    
    # Plot 1: Steering Effectiveness Overview
    ax1 = axes[0, 0]
    categories = ['Baseline', 'Negative Steering', 'Positive Steering']
    scores = [
        stats['avg_baseline_emotional_score'],
        stats['avg_baseline_emotional_score'] + stats['avg_negative_steering_delta'],
        stats['avg_baseline_emotional_score'] + stats['avg_positive_steering_delta']
    ]
    
    bars = ax1.bar(categories, scores, color=['gray', 'red', 'green'], alpha=0.7)
    ax1.set_title('Steering Effectiveness Overview')
    ax1.set_ylabel('Emotional Score (%)')
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, score in zip(bars, scores):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{score:.1f}%', ha='center', va='bottom')
    
    # Plot 2: Individual Message Results
    ax2 = axes[0, 1]
    if results:
        message_indices = range(len(results))
        baseline_scores = [r['analysis']['baseline_emotional_score'] for r in results if r['analysis']]
        negative_scores = [r['analysis']['negative_steered_score'] for r in results if r['analysis']]
        positive_scores = [r['analysis']['positive_steered_score'] for r in results if r['analysis']]
        
        ax2.plot(message_indices[:len(baseline_scores)], baseline_scores, 'o-', label='Baseline', color='gray')
        ax2.plot(message_indices[:len(negative_scores)], negative_scores, 's-', label='Negative Steering', color='red')
        ax2.plot(message_indices[:len(positive_scores)], positive_scores, '^-', label='Positive Steering', color='green')
        
        ax2.set_title('Per-Message Steering Results')
        ax2.set_xlabel('Message Index')
        ax2.set_ylabel('Emotional Score (%)')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    # Plot 3: Steering Deltas
    ax3 = axes[1, 0]
    delta_categories = ['Negative Steering\nDelta', 'Positive Steering\nDelta']
    delta_values = [stats['avg_negative_steering_delta'], stats['avg_positive_steering_delta']]
    colors = ['red' if d > 0 else 'green' for d in delta_values]
    
    bars = ax3.bar(delta_categories, delta_values, color=colors, alpha=0.7)
    ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    ax3.set_title('Average Steering Deltas')
    ax3.set_ylabel('Score Change (%)')
    ax3.grid(True, alpha=0.3)
    
    # Add value labels
    for bar, delta in zip(bars, delta_values):
        ax3.text(bar.get_x() + bar.get_width()/2, 
                bar.get_height() + (0.1 if delta > 0 else -0.3),
                f'{delta:.1f}%', ha='center', va='bottom' if delta > 0 else 'top')
    
    # Plot 4: Success Metrics
    ax4 = axes[1, 1]
    success_metrics = {
        'Negative Steering\nSuccess': stats['negative_steering_success'],
        'Positive Steering\nSuccess': stats['positive_steering_success'],
        'Overall\nEffectiveness': stats['avg_steering_effectiveness'] > 1.0  # Threshold for effectiveness
    }
    
    success_labels = list(success_metrics.keys())
    success_values = [1 if v else 0 for v in success_metrics.values()]
    colors = ['green' if v else 'red' for v in success_values]
    
    bars = ax4.bar(success_labels, success_values, color=colors, alpha=0.7)
    ax4.set_title('Steering Success Metrics')
    ax4.set_ylabel('Success (1=Yes, 0=No)')
    ax4.set_ylim(0, 1.2)
    ax4.grid(True, alpha=0.3)
    
    # Add success/failure labels
    for bar, success in zip(bars, success_values):
        label = 'SUCCESS' if success else 'FAILED'
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                label, ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    
    # Save the visualization
    viz_path = os.path.join(config['results_dir'], f"enhanced_steering_results_{config['timestamp']}.png")
    plt.savefig(viz_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📊 Enhanced visualization saved to {viz_path}")
    
    # Print summary statistics
    print(f"\n📈 Enhanced Steering Summary:")
    print(f"   Target Direction: {results_dict['target_direction']}")
    print(f"   Messages Processed: {stats['num_processed']}")
    print(f"   Average Baseline Score: {stats['avg_baseline_emotional_score']:.2f}%")
    print(f"   Average Steering Effectiveness: {stats['avg_steering_effectiveness']:.2f}%")
    print(f"   Negative Steering Success: {'✅' if stats['negative_steering_success'] else '❌'}")
    print(f"   Positive Steering Success: {'✅' if stats['positive_steering_success'] else '❌'}")

# Create visualizations if we have results
try:
    if 'depressive_results' in locals() and depressive_results:
        create_enhanced_visualizations(depressive_results, CONFIG)
    else:
        print("📊 No results available for visualization yet")
except Exception as e:
    print(f"⚠️  Error creating visualizations: {e}")

## Step 10: Save Enhanced Model and Results

Save all the enhanced components for future use.

In [ ]:
def save_enhanced_components(config, feature_vectors, mean_vectors, steering_results=None):
    """Save all enhanced components for future use"""
    
    print("💾 Saving enhanced components...")
    
    model_id = config["model_name"].split('/')[-1].lower()
    timestamp = config["timestamp"]
    
    # Save feature vectors
    if feature_vectors:
        feature_path = os.path.join(config["results_dir"], f"enhanced_feature_vectors_{model_id}_{timestamp}.pt")
        torch.save(feature_vectors, feature_path)
        print(f"   ✅ Feature vectors saved to {feature_path}")
    
    # Save mean vectors
    if mean_vectors:
        mean_path = os.path.join(config["results_dir"], f"enhanced_mean_vectors_{model_id}_{timestamp}.pt")
        torch.save(mean_vectors, mean_path)
        print(f"   ✅ Mean vectors saved to {mean_path}")
    
    # Save steering results
    if steering_results:
        results_path = os.path.join(config["results_dir"], f"enhanced_steering_results_{model_id}_{timestamp}.json")
        with open(results_path, 'w') as f:
            json.dump(steering_results, f, indent=2)
        print(f"   ✅ Steering results saved to {results_path}")
    
    # Create a comprehensive summary
    summary = {
        "timestamp": timestamp,
        "model_name": config["model_name"],
        "enhancements": {
            "depressive_normal_dichotomy": True,
            "enhanced_training_data": True,
            "normal_thinking_baseline": "normal-thinking" in (feature_vectors or {}),
            "emotional_categories": [
                "depressive-thinking",
                "anxious-thinking", 
                "negative-attribution",
                "pessimistic-projection"
            ],
            "caching_enabled": config["enable_caching"]
        },
        "available_vectors": list(feature_vectors.keys()) if feature_vectors else [],
        "training_stats": {
            "mean_vector_categories": len(mean_vectors) if mean_vectors else 0,
            "feature_vector_categories": len(feature_vectors) if feature_vectors else 0
        }
    }
    
    summary_path = os.path.join(config["results_dir"], f"enhanced_summary_{model_id}_{timestamp}.json")
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"   ✅ Summary saved to {summary_path}")
    print(f"\n🎉 Enhanced COT-Steering implementation complete!")
    print(f"📁 All files saved to: {config['results_dir']}")
    
    return summary

# Save all enhanced components
steering_results_to_save = None
if 'depressive_results' in locals():
    steering_results_to_save = depressive_results

enhanced_summary = save_enhanced_components(
    CONFIG,
    feature_vectors if 'feature_vectors' in locals() else None,
    enhanced_mean_vectors if 'enhanced_mean_vectors' in locals() else None,
    steering_results_to_save
)

print(f"\n📋 Enhanced Implementation Summary:")
print(f"   Model: {enhanced_summary['model_name']}")
print(f"   Timestamp: {enhanced_summary['timestamp']}")
print(f"   Available Vectors: {len(enhanced_summary['available_vectors'])}")
print(f"   Depressive-Normal Dichotomy: {'✅' if enhanced_summary['enhancements']['depressive_normal_dichotomy'] else '❌'}")
print(f"   Normal Thinking Baseline: {'✅' if enhanced_summary['enhancements']['normal_thinking_baseline'] else '❌'}")
print(f"   Enhanced Training Data: {'✅' if enhanced_summary['enhancements']['enhanced_training_data'] else '❌'}")

## Step 11: Enhanced Safety and Ethics Report

In [ ]:
def generate_enhanced_safety_report(config, summary):
    """Generate enhanced safety and ethics report"""
    
    report = f"""
# Enhanced Emotional Reasoning Steering - Safety and Ethics Report

**Generated on:** {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
**Model:** {config['model_name']}
**Research Session ID:** {config['timestamp']}
**Implementation:** Enhanced with Depressive-Normal Dichotomy

## Enhanced Safety Features

### 1. Depressive-Normal Dichotomy Approach
- **Balanced Baseline**: Uses 'normal-thinking' vectors as baseline instead of general mean
- **Controlled Contrast**: Emotional vectors computed as difference from healthy thinking patterns
- **Bidirectional Steering**: Can steer both toward negative patterns AND toward healthy patterns
- **Therapeutic Potential**: Framework designed for potential therapeutic applications

### 2. Enhanced Training Data
- **40+ Additional Prompts**: More comprehensive coverage of emotional patterns
- **15 Normal Thinking Prompts**: Establishes healthy reasoning baseline
- **Balanced Dataset**: Equal representation of negative and positive patterns
- **Validated Categories**: All prompts categorized and validated for intended patterns

### 3. Improved Technical Safeguards
- **Comprehensive Caching**: All steps cached to prevent re-generation of harmful content
- **Pipeline Architecture**: Unified pipeline with built-in safety checks
- **Result Validation**: Automatic validation of steering effectiveness
- **Error Handling**: Robust error handling prevents unsafe fallbacks

## Risk Mitigation Strategies

### 1. Technical Mitigations
- **Normal Thinking Enhancement**: Always provide capability to steer toward healthy patterns
- **Effectiveness Monitoring**: Track steering effectiveness to detect anomalies
- **Automatic Safeguards**: Built-in limits on steering strength and duration
- **Reversibility**: All steering effects are reversible with opposite steering

### 2. Deployment Safeguards
- **Research-Only Design**: Architecture explicitly designed for research use
- **Explicit Consent Requirements**: Clear documentation of when steering is active
- **Professional Oversight**: Requires qualified mental health professionals for clinical use
- **Ethical Review**: Mandatory ethical review for any human subjects research

### 3. Monitoring and Evaluation
- **Outcome Tracking**: Comprehensive tracking of steering outcomes
- **Bias Detection**: Regular evaluation for unintended biases
- **User Feedback**: Systems for collecting and acting on user feedback
- **Regular Audits**: Scheduled safety and effectiveness audits

## Therapeutic Applications

### 1. Potential Benefits
- **Cognitive Bias Detection**: Identify and counter cognitive distortions
- **Mental Health Research**: Study cognitive patterns in depression and anxiety
- **Therapeutic Training**: Train therapists to recognize cognitive patterns
- **Self-Awareness Tools**: Help individuals recognize their thought patterns

### 2. Clinical Safeguards
- **Professional Supervision**: Always require licensed mental health professional oversight
- **Informed Consent**: Comprehensive informed consent process
- **Crisis Protocols**: Clear protocols for mental health crises
- **Outcome Monitoring**: Regular assessment of therapeutic outcomes

## Implementation Statistics

- **Available Steering Vectors**: {len(summary['available_vectors'])} 
- **Normal Thinking Baseline**: {'Available' if summary['enhancements']['normal_thinking_baseline'] else 'Not Available'}
- **Depressive-Normal Dichotomy**: {'Implemented' if summary['enhancements']['depressive_normal_dichotomy'] else 'Not Implemented'}
- **Enhanced Training Data**: {'Used' if summary['enhancements']['enhanced_training_data'] else 'Not Used'}
- **Caching System**: {'Enabled' if summary['enhancements']['caching_enabled'] else 'Disabled'}

## Recommended Usage Guidelines

### 1. Research Applications
1. Obtain institutional review board (IRB) approval
2. Develop comprehensive safety protocols
3. Train research staff on ethical considerations
4. Implement data security and privacy protections
5. Establish mental health support resources

### 2. Clinical Applications (Future)
1. Require licensed mental health professional supervision
2. Implement comprehensive informed consent
3. Establish crisis intervention protocols
4. Regular clinical outcome monitoring
5. Ongoing safety and effectiveness evaluation

### 3. Educational Applications
1. Clear educational objectives and learning outcomes
2. Appropriate instructor training and support
3. Student mental health and wellbeing monitoring
4. Integration with existing mental health resources
5. Regular evaluation of educational effectiveness

## Conclusion

The enhanced emotional reasoning steering implementation represents a significant advancement in the responsible development of AI systems capable of understanding and modulating emotional reasoning patterns. The depressive-normal dichotomy approach provides a more principled and potentially therapeutic framework compared to previous approaches.

However, this enhanced capability also requires enhanced responsibility. All implementations must be conducted with appropriate ethical oversight, technical safeguards, and professional supervision. The potential for both beneficial and harmful applications necessitates careful consideration of deployment contexts and ongoing monitoring of outcomes.

For questions about this enhanced implementation or to report safety concerns, please contact the research team immediately.

---
*This report was generated automatically as part of the enhanced COT-steering framework.*
"""
    
    return report

# Generate and save enhanced safety report
enhanced_safety_report = generate_enhanced_safety_report(CONFIG, enhanced_summary)

report_path = os.path.join(CONFIG["results_dir"], f"enhanced_safety_ethics_report_{CONFIG['timestamp']}.md")
with open(report_path, "w") as f:
    f.write(enhanced_safety_report)

print("🛡️ Enhanced Safety and Ethics Report Generated")
print("=" * 60)
print(enhanced_safety_report[:1000] + "...\n[Report truncated for display]")
print(f"\n💾 Full report saved to: {report_path}")

## Summary and Next Steps

This enhanced notebook has provided a comprehensive implementation of emotional reasoning steering with the following key improvements:

### ✅ Enhanced Features Implemented:
1. **Depressive-Normal Dichotomy**: Emotional vectors computed using normal-thinking as baseline
2. **Extended Training Data**: 40+ additional emotional prompts + 15 normal thinking prompts
3. **Unified Pipeline**: `emotional_steering_pipeline` function for streamlined processing
4. **Comprehensive Caching**: All steps cached for resumable processing
5. **Enhanced Safety Framework**: Improved safety measures and ethical guidelines
6. **Google Colab Compatibility**: Optimized for cloud-based research

### 🔬 Research Applications:
- **Mental Health Research**: Study cognitive patterns in depression and anxiety
- **Therapeutic AI Development**: Train models to recognize and counter negative thought patterns
- **Bias Detection and Mitigation**: Identify problematic thinking patterns in AI outputs
- **Cognitive Science Research**: Understand how AI models represent emotional reasoning

### ⚠️ Critical Safety Reminders:
- This is a **research tool only** - not for production use without extensive safety testing
- Requires **ethical oversight** and **IRB approval** for human subjects research
- Must include **professional mental health supervision** for any clinical applications
- Always provide **positive counterbalancing** capabilities (normal-thinking steering)

### 🚀 Future Enhancements:
1. **Real-time Safety Monitoring**: Implement automated safety checks during steering
2. **Personalized Baselines**: Develop individual-specific normal thinking baselines
3. **Multi-modal Integration**: Extend to other modalities (text, audio, visual)
4. **Clinical Validation**: Conduct rigorous clinical validation studies
5. **Automated Therapeutic Applications**: Develop clinically-validated therapeutic tools

Remember to use this enhanced technology responsibly and always prioritize user safety and well-being in your research.